In [1]:
import ngmix
import numpy as np
import joblib

In [30]:

def gen_data(n, seed, sigma_e=0.2, g_true_mn=0.02, R=0.5, g_true_sd=0.01):
    rng = np.random.RandomState(seed=seed)
    e = rng.normal(size=n, scale=sigma_e)
    g_true = rng.normal(size=n, scale=g_true_sd, loc=g_true_mn)
    g = e + R*g_true
    return g, R

def weight_fun(g):
    prior = ngmix.priors.GPriorBA(0.3, rng=np.random.RandomState())
    pvals = prior.get_prob_array2d(g, 0)
    return pvals

def meas_data_mn(g, R):
    w = weight_fun(g)
    
    g_p = g + R*0.01
    w_p = weight_fun(g_p)
    
    g_m = g - R*0.01
    w_m = weight_fun(g_m)
    
    return np.sum(w*g*w*g), np.sum(w), np.sum(w_p * g_p), np.sum(w_p), np.sum(w_m * g_m), np.sum(w_m)

In [31]:
def _fun(n, seed):
    g, R = gen_data(n, seed)
    return meas_data_mn(g, R)

In [32]:
n_tot = 10_000_000
n_jobs = 2000
n_per = n_tot // n_jobs

seeds = np.random.RandomState(seed=32).randint(1, 2**30, size=n_jobs)

jobs = [
    joblib.delayed(_fun)(n_per, seed)
    for seed in seeds
]

with joblib.Parallel(n_jobs=-1, verbose=100) as par:
    all_sums = par(jobs)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0679s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  14 ta

[Parallel(n_jobs=-1)]: Done 1403 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1406 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1409 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1412 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1415 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1447 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1482 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1514 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1517 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1520 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1523 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1526 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1529 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1561 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1564 tasks      | elapsed: 

In [33]:
def _jk_est(all_sums):
    all_sums = np.array(all_sums)
    n = all_sums.shape[0]
    gjk = np.zeros(n)
    inds = np.arange(n)
    for i in range(n):
        msk = inds != i
        sums = np.sum(all_sums[msk, :], axis=0)
        g1 = sums[0]/sums[1]
        g1_1p = sums[2]/sums[3]
        g1_1m = sums[4]/sums[5]
        R11 = (g1_1p - g1_1m)/0.02

        gjk[i] = g1/R11/R11
        
    mn = np.mean(gjk)
    dgjk = gjk - mn
    sd = np.sqrt((n-1)/n*np.sum(dgjk**2))
    return mn, sd

In [34]:
g2_mn, g2_sd = _jk_est(all_sums)

In [35]:
tvar = (0.2**2 + 0.5**2 * (0.02**2 + 0.01**2))/0.5**2

m_mn, m_sd = g2_mn/tvar-1, g2_sd/tvar

print("var(g) frac err = %0.3f +/- %0.3f [10^-3, 3sigma]" % (
    m_mn/1e-3,
    m_sd*3/1e-3
))

var(g) frac err = 617.863 +/- 3.823 [10^-3, 3sigma]
